In [7]:
import talib as ta
from datetime import datetime
from datetime import timedelta
from bigtrader.constant import Direction, OrderType

STRATEGY_NAME = "STRATEGY_1m"

# 第一步：获取历史数据，计算唐奇安通道和ATR
# 第二步：当突破唐奇安通道时，开仓。
# 第三步：计算加仓和止损信号。
# 回测初始资金：1000万

def initialize(context):
    """初始化"""
    print("initialize")  
#     context.symbol = instruments
#     context.count = 0
    context.my_instruments = context.get_conf_param("instruments")#从传入参数中获取需要交易的合约
    context.closetime_day = context.get_conf_param("closetime_day")#日内策略白盘平仓时间，一般14:58
    context.closetime_night = context.get_conf_param("closetime_night")#日内策略夜盘平仓时间，一般22:58，注意有些商品夜盘收盘时间不一样
    context.order_num = context.get_conf_param("order_num")#下单手数
    context.set_universe(context.my_instruments)#设置需要处理的合约
    # 设置计算唐奇安通道的参数
    context.n = 20
#     # 设置交易最大资金比率，这里设置在volume_limit
#     context.ratio = 0.8


def before_trading(context, data):
    """盘前处理"""
    #print("before_trading")
    context.subscribe(context.my_instruments) #注册合约
    context.index = 0
    context.max_high = 0
    context.max_low = 0

    
def handle_data(context, data):
    """Bar行情推送"""
    cur_date =  data.current_dt
    cur_hm = cur_date.strftime('%H:%M') #time     
    #当日最高最低
    today_high = data.current(context.my_instruments,'high')
    today_low = data.current(context.my_instruments,'low')
    context.max_high = max(context.max_high,today_high)
    context.max_low = max(context.max_high,today_low) 
    
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(context.my_instruments, Direction.LONG)
    position_short = context.get_position(context.my_instruments, Direction.SHORT)
    # 获取当前价格
    price = data.current(context.my_instruments, "close")
    
    #部分品种夜盘收盘时间不一样，此时间表示指定的尾盘平仓时间往后偏移30分钟，这段时间内不能开新仓，只能平仓。给30分钟是为了足够的冗余
    closetime_nightshift = (datetime.strptime(context.closetime_night,'%H:%M') + timedelta(minutes = 30)).strftime('%H:%M')  
    #尾盘平仓
    if((cur_hm>=context.closetime_day and cur_hm<="15:00") or (cur_hm>=context.closetime_night and cur_hm<=closetime_nightshift)):
        if(position_long.current_qty != 0):
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        if(position_short.current_qty != 0):
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        #尾盘不开新仓，直接返回
        return
  
    #交易逻辑
    
    # 计算通道的数据:当日最低、最高、上一交易日收盘
    # 注：由于talib库计算ATR的结果与公式求得的结果不符，所以这里利用公式计算ATR
    
    data_for_ATR = data.history(context.my_instruments, ["close","high","low"], context.n, "1d") # 计算ATR
    tr_list = []
    for i in range(1, len(data_for_ATR)):
        tr = max((data_for_ATR['high'].iloc[i] - data_for_ATR['low'].iloc[i]),
                 data_for_ATR['close'].shift(-1).iloc[i] - data_for_ATR['high'].iloc[i],
                 data_for_ATR['close'].shift(-1).iloc[i] - data_for_ATR['low'].iloc[i])
        tr_list.append(tr)
    # 把最新一期tr（当日最高最低）加入列表中
    tr_new = max((context.max_high - context.max_low),
                 data_for_ATR['close'].iloc[-1] - context.max_high,
                 data_for_ATR['close'].iloc[-1] - context.max_low)
    tr_list.append(tr_new)
    context.atr = int(np.floor(np.mean(tr_list)))
    context.atr_half = int(np.floor(0.5 * context.atr))
    # 计算唐奇安通道
    context.don_open = np.max(data_for_ATR['high'].values[-context.n:])
    context.don_close = np.min(data_for_ATR['low'].values[-context.n:])
        
    # 计算加仓点和止损点
    context.long_add_point = context.don_open + context.atr_half
    context.long_stop_loss = context.don_open - context.atr_half
    context.short_add_point = context.don_close - context.atr_half
    context.short_stop_loss = context.don_close + context.atr_half
    
    
    # 当无持仓时
    if position_long.current_qty == 0 and position_short.current_qty == 0:
        # 如果向上突破唐奇安通道，则开多 （这里的下单是atr为单位）
        if price > context.don_open:
            rv = context.buy_open(context.my_instruments, context.atr, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
        # 如果向下突破唐奇安通道，则开空
        if price < context.don_close:
            rv = context.sell_open(context.my_instruments, context.atr, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1)   
    # 有持仓时
    # 持多仓，继续突破（加仓）
    if position_long.current_qty != 0:
        # 当突破1/2atr时加仓
        if price > context.long_add_point:
            rv = context.buy_open(context.my_instruments, context.atr_half, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 继续加多单 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
            # 上调点位
            context.long_add_point += context.atr_half
            context.long_stop_loss += context.atr_half
        # 平仓逻辑
        if price < context.long_stop_loss:
            if position_long.current_qty >= context.atr_half:
                rv = context.sell_close(context.my_instruments, context.atr_half, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 当前持仓大于0.5atr,平0.5atr for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) 
            else:
                rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 当前持仓小于0.5atr,平当前全部的持仓 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) 
            print('平多仓0.5atr')
            context.long_add_point -= context.atr_half
            context.long_stop_loss -= context.atr_half
    # 持空仓，继续突破（加仓）
    if position_short.current_qty != 0:
        # 当跌破加仓点时加仓
        if price < context.short_add_point:
            rv = context.sell_open(context.my_instruments, context.atr_half, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 继续加空单 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
            context.short_add_point -= context.atr_half
            context.short_stop_loss -= context.atr_half
        # 持多仓，止损位计算
        if price > context.short_stop_loss:
            if position_short.current_qty >= context.atr_half:
                rv = context.buy_close(context.my_instruments, context.atr_half, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 当前持仓大于0.5atr,平0.5atr for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) 
            else:
                rv = context.buy_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
                msg = str(data.current_dt) +  " 当前持仓小于0.5atr,平当前全部的持仓 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                context.write_log(msg, stdout=1) 
            print('平空仓0.5atr')
            context.short_add_point += context.atr_half
            context.short_stop_loss += context.atr_half


            
def handle_order(context, order):
    """委托回报推送"""
    msg = "handle_order:" + order.log_str()
    context.write_log(msg, stdout=1) 
         
def handle_trade(context, trade):
    """成交回报推送"""
    msg = "handle_trade:" + trade.log_str()
    context.write_log(msg, stdout=1) 
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(trade.symbol, Direction.LONG)
    position_short = context.get_position(trade.symbol, Direction.SHORT)
    msg = "当前多头持仓:"+ str(position_long) + "当前空头持仓:"+ str(position_short)
    context.write_log(msg, stdout=1) 

instruments = "RB2110.SHF" #["RB2110.SHF"] #["RB2110.SHF"] [I2105.DCE"] ['RB']# RB2101.SHF 如果想用主连合约，可以出入合约简码，例如RB,I,CF

#需要交易者传入的参数
strategy_setting = [
    {
        "instruments": instruments,
#         "order_num": 1,                                       
        "closetime_day": "14:58",
        "closetime_night": "22:58"
    }    
    
]
start_date = "2021-01-06"
end_date = "2021-04-06"
md = M.hfbacktest.v1(start_date=start_date,
                     end_date=end_date,
                     instruments=[instruments], #只传入一个合约便于策略逻辑展示
                     capital_base=10000000, #为了进行仓位管理，将资金量调整更大一点，并约定每次下单头寸是账户的一个atr，加仓是半个atr，也可以更换仓位管理思路，用1%的头寸作为下单单位
                     product_type=Product.FUTURE,
                     frequency=Frequency.MINUTE,
                     initialize=initialize,
                     before_trading_start=before_trading,
                     handle_data=handle_data,
                     handle_order=handle_order,
                     handle_trade=handle_trade,
                     plot_charts=True,
                     volume_limit=0.8,
                     disable_cache=0,
                     show_debug_info=1,
                     strategy_setting=strategy_setting,
                     slippage_type=SlippageType.FIXED,#滑点固定模式
                     slippage_value=1.0,#买卖双向各1个滑点
                     m_deps=np.random.rand())



[2021-05-24 16:08:01.403806] INFO: moduleinvoker: hfbacktest.v1 开始运行..

[2021-05-24 16:08:01.410596] INFO: hfbacktest: passed-in daily_data_ds:None

[2021-05-24 16:08:01.411834] INFO: hfbacktest: passed-in minute_data_ds:None

[2021-05-24 16:08:01.412818] INFO: hfbacktest: passed-in tick_data_ds:None

[2021-05-24 16:08:01.413623] INFO: hfbacktest: passed-in each_data_ds:None

...


2021-05-24 16:11:16.690012 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5073.0,MARKET,strategy_20210524), available:109713.17999997895
2021-05-24 16:11:16.690200 strategy_20210524(bkt999,): 2021-04-02 14:30:00 继续加多单 for RB2110.SHF 最新价=5073.0 rv:-114 
2021-05-24 16:11:16.698398 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5074.0,MARKET,strategy_20210524), available:109713.17999997895 
2021-05-24 16:11:16.698736 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5074.0,MARKET,strategy_20210524), available:109713.17999997895
2021-05-24 16:11:16.698907 strategy_20210524(bkt999,): 2021-04-02 14:31:00 继续加多单 for RB2110.SHF 最新价=5074.0 rv:-114 
2021-05-24 16:11:16.707247 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5085.0,MARKET,strategy_20210524), available:109713.17

2021-05-24 16:11:16.908656 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5090.0,MARKET,strategy_20210524), available:109713.17999997895 
2021-05-24 16:11:16.908800 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5090.0,MARKET,strategy_20210524), available:109713.17999997895
2021-05-24 16:11:16.908927 strategy_20210524(bkt999,): 2021-04-02 14:50:00 继续加多单 for RB2110.SHF 最新价=5090.0 rv:-114 
2021-05-24 16:11:16.922335 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5088.0,MARKET,strategy_20210524), available:109713.17999997895 
2021-05-24 16:11:16.922486 handle_order_submit process req check cash failed for order_req:OrderReq(bkt999,RB2110.SHF,long,open,60,5088.0,MARKET,strategy_20210524), available:109713.17999997895
2021-05-24 16:11:16.922864 strategy_20210524(bkt999,): 2021-04-02 14:51:00 继续加多单 for RB2110.SHF 最新价=5088.0

2021-05-24 16:11:19.206171 strategy_20210524(bkt999,): handle_order:[bkt999,673,RB2110.SHF,short,close_today,0,129,5148.0,pending,14:58:00,673] 
2021-05-24 16:11:19.206455 strategy_20210524(bkt999,): 2021-04-06 14:58:00 尾盘平多 for RB2110.SHF 最新价=5148.0 rv:0 
2021-05-24 16:11:19.208409 strategy_20210524(bkt999,): handle_order:[bkt999,673,RB2110.SHF,short,close_today,129,129,5148.0,filled,14:58:00,673] 
2021-05-24 16:11:19.209254 strategy_20210524(bkt999,): handle_trade:[bkt999,673,RB2110.SHF,short,close_today,129,5147.0,SHFE.669,14:59:00.000000] 
2021-05-24 16:11:19.209359 strategy_20210524(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:5120.0,last_price:5147.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-24 16:11:19.223169 run_backtest() finished! time cost 193.715s! 


[2021-05-24 16:11:20.319824] INFO: hfbacktest: backtest done, raw_perf_ds:DataSource(e66aa9f40475474c8d32890d25e8a5d4T)

[2021-05-24 16:11:20.944836] INFO: moduleinvoker: hfbacktest.v1 运行完成[199.541024s].